<a href="https://colab.research.google.com/github/Sreekar-Kandhadai/pyspark-interview-questions/blob/main/Time_Spent(Login_Logout).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Calculate how much time each user spends between log in and log out.
Find the difference between the logout_timestamp and login_timestamp in hours, minutes, and seconds.
The result should be formatted like "HH:mm:ss".

In [14]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

spark=SparkSession.builder.appName("learning").getOrCreate()

data = [ (1, "2025-01-31 08:00:00", "2025-01-31 10:30:45"),
(2, "2025-01-31 09:00:30", "2025-01-31 12:15:10"),
(3, "2025-01-31 07:45:00", "2025-01-31 09:00:15") ]

schema_columns = ["user_id", "login_timestamp", "logout_timestamp"]

df=spark.createDataFrame(data, schema_columns)

df.show()

df1=df.withColumn("login_time",unix_timestamp(col('login_timestamp')))\
      .withColumn("logout_time",unix_timestamp(col('logout_timestamp')))\
      .withColumn("duration_seconds",col("logout_time")-col("login_time"))

df1.show()

# df2=df1.withColumn("hour",floor(col("duration_seconds")/3600))

df2=df1.withColumn("hour",(col("duration_seconds")/3600).cast("int"))\
       .withColumn("minute",((col("duration_seconds")-(col("hour")*3600))/60).cast("int"))\
       .withColumn("second",(col('duration_seconds')-(col('hour')*3600)- (col('minute')*60)).cast("int"))\
       .withColumn("time_spent_by_each_user",concat_ws(':',col('hour'),col('minute'),col('second')))\
       .select("user_id","login_timestamp","logout_timestamp","time_spent_by_each_user")

df2.show()

+-------+-------------------+-------------------+
|user_id|    login_timestamp|   logout_timestamp|
+-------+-------------------+-------------------+
|      1|2025-01-31 08:00:00|2025-01-31 10:30:45|
|      2|2025-01-31 09:00:30|2025-01-31 12:15:10|
|      3|2025-01-31 07:45:00|2025-01-31 09:00:15|
+-------+-------------------+-------------------+

+-------+-------------------+-------------------+----------+-----------+----------------+
|user_id|    login_timestamp|   logout_timestamp|login_time|logout_time|duration_seconds|
+-------+-------------------+-------------------+----------+-----------+----------------+
|      1|2025-01-31 08:00:00|2025-01-31 10:30:45|1738310400| 1738319445|            9045|
|      2|2025-01-31 09:00:30|2025-01-31 12:15:10|1738314030| 1738325710|           11680|
|      3|2025-01-31 07:45:00|2025-01-31 09:00:15|1738309500| 1738314015|            4515|
+-------+-------------------+-------------------+----------+-----------+----------------+

+-------+---------